In [5]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
df = pd.read_csv('data/vgsales.csv')

# Data Preprocessing
# Fill missing values in 'Year' with the median year
df['Year'] = df['Year'].fillna(df['Year'].median())

# Encoding categorical variables like 'Platform', 'Genre', and 'Publisher'
label_encoders = {}
for column in ['Platform', 'Genre', 'Publisher']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column].astype(str))
    label_encoders[column] = le

# Defining the feature set and the target variable
X = df[['Platform', 'Year', 'Genre', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']]
y = df['Global_Sales']

# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Function to evaluate models
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    print(f"RMSE: {rmse}")
    print(f"R-squared: {r2}")
    return y_pred

# 1. Linear Regression
print("Linear Regression:")
lr = LinearRegression()
lr.fit(X_train, y_train)
evaluate_model(lr, X_test, y_test)

# 2. Decision Tree
print("\nDecision Tree:")
dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_train, y_train)
evaluate_model(dt, X_test, y_test)

# 3. Random Forest
print("\nRandom Forest:")
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
evaluate_model(rf, X_test, y_test)

# 4. XGBoost
print("\nXGBoost:")
xgb = XGBRegressor(n_estimators=100, random_state=42)
xgb.fit(X_train, y_train)
evaluate_model(xgb, X_test, y_test)


Linear Regression:
RMSE: 0.005187915357340992
R-squared: 0.9999916063331278

Decision Tree:
RMSE: 0.7295054064508728
R-squared: 0.8340325218394546

Random Forest:
RMSE: 0.7121855230914278
R-squared: 0.841819753495406

XGBoost:
RMSE: 0.7786799676398508
R-squared: 0.8109032913868871


array([0.16687323, 0.39581677, 0.02217262, ..., 0.918934  , 0.05432508,
       0.21571656], dtype=float32)